In [2]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [3]:
X = []
Y = []
All = []
with open('./src/train_simple_reg.csv', 'r') as f:
    print(f.readline())
    for line in f:
        data = line.split(',')
        _id = int(data[0])
        y = float(data[-1])
        x = [float(_x) for _x in data[1:-1]]
        
        X.append(x)
        Y.append(y)
        
        all_data = [float(_x) for _x in data[1:]]
        
        All.append(all_data)
        


id,f1,f2,f3,f4,f5,y



In [4]:
X = np.array(X)
Y = np.array(Y)
All = np.array(All)
test = All[:2]
np.linalg.matrix_rank(test)
W = np.linalg.lstsq(X, Y, rcond=None)[0]

In [5]:
# W=np.array([-5.023425,25.22800208,-10.11129295,-8.66006065,31.18213141])

In [6]:
myY = X.dot(W)
myY

array([  74.54866594,   14.50572028, -174.8535377 , ...,    4.50980975,
         90.60301226,  101.8507598 ])

In [7]:
((X.dot(W) - Y)**2).mean()**0.5

0.30380527811683417

In [8]:
Xtest = []
Ytest = []
ID = []
with open('./src/test_data_simple_reg.csv', 'r') as f:
    print(f.readline())
    for line in f:
        data = line.split(',')
        _id = int(data[0])
        y = float(data[-1])
        x = [float(_x) for _x in data[1:]]
        
        ID.append(_id)
        Xtest.append(x)


id,f1,f2,f3,f4,f5



In [9]:
Xtest = np.array(Xtest)
Ytest = []

In [10]:
Xtest

array([[ 0.2792066 ,  1.16249803,  2.30958627, -1.8945449 , -0.59326735],
       [-0.8004779 , -1.30670378,  0.26431358,  1.04567154, -0.34619676],
       [ 0.26461917,  2.72806125, -0.57301071,  0.1294317 ,  0.31849361],
       ...,
       [-1.80250274,  1.21102754, -0.06093642, -0.66868357,  0.33629607],
       [-0.53912127,  0.02144329, -1.07816718,  0.2168838 ,  0.05240968],
       [-0.53737587, -0.11286944, -0.82219981, -0.23054688, -0.74382173]])

In [19]:
Ytest = Xtest.dot(W)
ID[0]

0

In [32]:
Ytest = np.array(Ytest)
Ytest.shape
result = []
for idx,line in enumerate(Ytest):
    result.append([idx, line])
    

In [33]:
np.savetxt("random_sol.csv", result, fmt="%i,%f",  delimiter=',', header="id,y", comments='')

PermissionError: [Errno 13] Permission denied: 'random_sol.csv'